In [1]:
from scipy import sparse
import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)
import BCH_BSC_Functions as func

In [2]:
seed=16

In [3]:
S=32
n=20
BCH_Poly = 37
Bits_per_error=5
samples=10000
p=0.05
M=32
test_size=20000
n_state=7
n_message=18

codes_combined=func.code_combined(S,M,n,BCH_Poly,Bits_per_error)
#codes_nested=func.code_nested(S,M,n,BCH_Poly,Bits_per_error,10)
#codes_sep_state,codes_sep_message=func.code_separate(S,M,n_state,n_message,BCH_Poly,Bits_per_error)
codes_punctured=func.code_punctured(S,M,n,BCH_Poly,Bits_per_error)

#Generate random sparse transition matrix with density=(1-sparsity)
np.random.seed(seed)
trans_mat=sparse.rand(S,S,density=0.125)
trans_mat=trans_mat.todense()
num_of_trans=np.sum(trans_mat!=0,axis=1)
for i in range(0,S):
    if(num_of_trans[i]==0):
        trans_mat[i,:]=sparse.rand(1,S,density=0.125).todense()
trans_mat=trans_mat/np.sum(trans_mat,axis=1)

In [4]:
#Online learning - No separate training phase, transition matrix is learnt along with decoding

In [5]:
np.random.seed(seed)
states,messages=func.data_generator(M,trans_mat,test_size)

In [6]:
sent_punctured=func.encoder(M,states,messages,codes_punctured,test_size)
np.random.seed(seed)
received_punctured=func.channel(sent_punctured,test_size,p)

In [7]:
state_predicted,message_predicted=func.decoder_online_learning(S,M,n,received_punctured,p,codes_punctured,20000,"HMM_delay_MAP",1)
print(func.errors(states,messages,state_predicted,message_predicted,test_size))
print(func.errors(states[0:10000],messages[0:10000],state_predicted[0:10000],message_predicted[0:10000],10000))
print(func.errors(states[10000:20000],messages[10000:20000],state_predicted[10000:20000],message_predicted[10000:20000],10000))

(50, 76)
(29, 39)
(21, 37)


In [8]:
#A separate training phase and then testing separately

In [9]:
np.random.seed(seed)
p_predicted,trans_mat_predicted=func.training(S,n,BCH_Poly,Bits_per_error,samples,p,trans_mat)

In [10]:
#Using the original p and transition matrix
state_predicted,message_predicted=func.decoder(S,M,n,received_punctured,trans_mat,p,codes_punctured,test_size,"HMM_delay_MAP",1)
print(func.errors(states,messages,state_predicted,message_predicted,test_size))
print(func.errors(states[0:10000],messages[0:10000],state_predicted[0:10000],message_predicted[0:10000],10000))
print(func.errors(states[10000:20000],messages[10000:20000],state_predicted[10000:20000],message_predicted[10000:20000],10000))

(35, 61)
(17, 27)
(18, 34)


In [11]:
#Using the learnt p and transition matrix
state_predicted,message_predicted=func.decoder(S,M,n,received_punctured,trans_mat_predicted,p_predicted,codes_punctured,test_size,"HMM_delay_MAP",1)
print(func.errors(states,messages,state_predicted,message_predicted,test_size))
print(func.errors(states[0:10000],messages[0:10000],state_predicted[0:10000],message_predicted[0:10000],10000))
print(func.errors(states[10000:20000],messages[10000:20000],state_predicted[10000:20000],message_predicted[10000:20000],10000))

(35, 61)
(16, 26)
(19, 35)
